# Fill the shared array with data

In [1]:
import numpy as np
import skmap_bindings
import time
import tempfile
from pathlib import Path

def _model_input(tile, start_year = 2000, end_year = 2022, bands = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2', 'thermal'], base_url='http://192.168.49.30:8333'):
    prediction_layers = []

    for year in range(start_year, end_year + 1):
        for band in bands:
            prediction_layers += [
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0101_{year}0228_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0301_{year}0430_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0501_{year}0630_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0701_{year}0831_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}0901_{year}1031_go_epsg.4326_v20230908.tif',
            f'{base_url}/prod-landsat-ard2/{tile}/seasconv/{band}_glad.SeasConv.ard2_m_30m_s_' + f'{year}1101_{year}1231_go_epsg.4326_v20230908.tif'
            ]

    return prediction_layers

In [2]:
raster_files = _model_input("013E_61N", start_year = 1997, end_year = 2001, bands = ['blue'], base_url='http://192.168.1.30:8333')
hosts = [ f'192.168.1.{i}:8333' for i in range(30,43) ]
raster_files = [str(r).replace("192.168.1.30", f"192.168.1.{30 + int.from_bytes(Path(r).stem.encode(), 'little') % len(hosts)}") for r in raster_files]

n_feat = len(raster_files)
n_threads = 48
x_off = 2
y_off = 2
x_size = 4000
y_size = 4000

bands_list = [1,] # The first band is indexed as 1

conf_GDAL = {
             "CPL_VSIL_CURL_ALLOWED_EXTENSIONS": ".tif",
             "GDAL_HTTP_MULTIRANGE": "SINGLE_GET",
             "GDAL_HTTP_CONNECTTIMEOUT": "320",
             "GDAL_HTTP_TIMEOUT": "320",
             "CPL_VSIL_CURL_USE_HEAD": "NO",
             "GDAL_HTTP_VERSION": "1.0",
             "CPL_CURL_GZIP": "NO"}

perm_vec = range(n_feat) # row indices of the array where each tiff contenct will go (starting from first element of file_urls)

In [3]:
data = np.empty((n_feat,x_size*y_size), dtype=np.float32, order='C')
print("Starting")
start = time.time()
skmap_bindings.parReadFiles(data, n_threads, raster_files, perm_vec, x_off, y_off, x_size, y_size, bands_list, conf_GDAL)
print(f"C++ part done in {time.time() - start:.2f} s")
print(data)
print(data.shape)

Starting
C++ part done in 1.05 s
[[255. 255. 255. ... 255. 255. 255.]
 [255. 255. 255. ... 255. 255.   5.]
 [  8.   9.   9. ...   6.   6.   5.]
 ...
 [  4.   4.   4. ...   5.   5.   5.]
 [ 28.  30.  30. ...   4.   4.   5.]
 [ 28.  30.  30. ...   4.   4.   5.]]
(30, 16000000)
